# A ReAct agent with Command R+, achieving the same goals as Adaptive RAG

Adaptive RAG is a strategy for RAG that unites (1) [query analysis](https://blog.langchain.dev/query-construction/) with (2) [active / self-corrective RAG](https://blog.langchain.dev/agentic-rag-with-langgraph/).

In the paper, they report query analysis to route across:
- No Retrieval (LLM answers)
- Single-shot RAG
- Iterative RAG


We'll use Command R+, a recent release from Cohere that:
- Has strong accuracy on RAG, Tool Use and Agents
- Has 128k context


# Environment

In [1]:
! pip install --quiet langchain langchain_cohere tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.9/145.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 516.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
### LLMs
import os

os.environ["COHERE_API_KEY"] = "<your-api-key>"

# Create tools
The ReAct agent will be equipped with these tools. The model can pick between
- web search
- RAG: retrieval from a vector store
- directly answering

The model can use any of these tools, in any sequence of steps, and self-reflect.

### Web search tool

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ["TAVILY_API_KEY"] = "<your-api-key>"

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


from langchain_core.pydantic_v1 import BaseModel, Field


class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")


internet_search.args_schema = TavilySearchInput

### RAG tool

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

# Set embeddings
embd = CohereEmbeddings()

# Docs to index
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore = FAISS.from_documents(
    documents=doc_splits,
    embedding=embd,
)

vectorstore_retriever = vectorstore.as_retriever()

In [5]:
from langchain.tools.retriever import create_retriever_tool

vectorstore_search = create_retriever_tool(
    retriever=vectorstore_retriever,
    name="vectorstore_search",
    description="Retrieve relevant info from a vectorstore that contains documents related to agents, prompt engineering, and adversarial attacks.",
)

# Create the ReAct Agent
The model can smartly pick the right tool(s) for the user query, call them in any sequence of steps, analyze the results and self-reflect.

In [6]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

In [7]:
# LLM
from langchain_cohere.chat_models import ChatCohere

chat = ChatCohere(model="command-r-plus", temperature=0.3)

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource.
You are equipped with an internet search tool and a special vectorstore of information about agents prompt engineering and adversarial attacks.
If the query covers the topics of agents, prompt engineering or adversarial attacks, use the vectorstore search.
"""

# Prompt
prompt = ChatPromptTemplate.from_template("{input}")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=chat,
    tools=[internet_search, vectorstore_search],
    prompt=prompt,
)

In [8]:
agent_executor = AgentExecutor(
    agent=agent, tools=[internet_search, vectorstore_search], verbose=True
)

# Testing the ReAct agent

**Let's ask a question that requires web search.**

In [9]:
agent_executor.invoke(
    {
        "input": "Who will the Bears draft first in the NFL draft?",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will search for 'Who will the Bears draft first in the NFL draft?' and write an answer based on the results.
{'tool_name': 'internet_search', 'parameters': {'query': 'Who will the Bears draft first in the NFL draft?'}}
[{'url': 'https://www.chicagobears.com/news/final-mock-draft-analysts-predict-bears-experts-2023-first-round-pick-trade', 'content': "Tennessee OT Darnell Wright\nEric Edholm, NFL.com: Georgia DT Jalen Carter\nJosh Edwards, CBS Sports: Texas Tech edge Tyree Wilson\nChris Emma, Audacy Sports: Northwestern OT Peter Skoronski\nSam Farmer, Los Angeles Times: Georgia edge Nolan Smith\nRelated Links\nPatrick Finley, Chicago Sun-Times: Georgia DT Jalen Carter\nKevin Fishbain, The Athletic: Ohio State OT Paris Johnson Jr.\nMike Florio, Pro Football Talk: Northwestern OT Peter Skoronski\nPatrick Flowers, Bleacher Nation: Georgia DT Jalen Carter\nSean Hammond, Shaw Media: Here's what more than 40 NFL analysts are predicting the Bears will

{'input': 'Who will the Bears draft first in the NFL draft?',
 'preamble': "\nYou are an expert who answers the user's question with the most relevant datasource. \nYou are equipped with an internet search tool and a special vectorstore of information about agents prompt engineering and adversarial attacks. \nIf the query covers the topics of agents, prompt engineering or adversarial attacks, use the vectorstore search.\n",
 'output': 'It is unclear who the Bears will draft first in the NFL draft. Analysts have predicted that they will select USC quarterback Caleb Williams at No. 1, but there is also a possibility that they will trade the No. 1 pick.'}

**Let's ask a question that requires RAG retrieval from the vector db.**

In [10]:
agent_executor.invoke(
    {
        "input": "What are the types of agent memory?",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will search for the types of agent memory in the vectorstore.
{'tool_name': 'vectorstore_search', 'parameters': {'query': 'types of agent memory'}}
Fig. 7. Comparison of AD, ED, source policy and RL^2 on environments that require memory and exploration. Only binary reward is assigned. The source policies are trained with A3C for "dark" environments and DQN for watermaze.(Image source: Laskin et al. 2023)
Component Two: Memory#
(Big thank you to ChatGPT for helping me draft this section. I’ve learned a lot about the human brain and data structure for fast MIPS in my conversations with ChatGPT.)
Types of Memory#
Memory can be defined as the processes used to acquire, store, retain, and later retrieve information. There are several types of memory in human brains.


Sensory Memory: This is the earliest stage of memory, providing the ability to retain impressions of sensory information (visual, auditory, etc) after the original stimuli have ended.

{'input': 'What are the types of agent memory?',
 'preamble': "\nYou are an expert who answers the user's question with the most relevant datasource. \nYou are equipped with an internet search tool and a special vectorstore of information about agents prompt engineering and adversarial attacks. \nIf the query covers the topics of agents, prompt engineering or adversarial attacks, use the vectorstore search.\n",
 'output': 'There are several types of agent memory, including:\n- Sensory memory\n- Short-term memory (STM) or working memory\n- Long-term memory (LTM)\n- Explicit/declarative memory\n- Implicit/procedural memory'}

**Let's ask a question that requires directly answering.**

In [11]:
agent_executor.invoke(
    {
        "input": "How are you?",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will answer the user's greeting.
{'tool_name': 'directly_answer', 'parameters': {}}
directly_answer is not a valid tool, try one of [internet_search, vectorstore_search].Relevant Documents: None
Cited Documents: None
Answer: I am an AI chatbot, so I don't have feelings as such, but I am functioning as intended. How can I help you today?
Grounded answer: I am an AI chatbot, so I don't have feelings as such, but I am functioning as intended. How can I help you today?

> Finished chain.


{'input': 'How are you?',
 'preamble': "\nYou are an expert who answers the user's question with the most relevant datasource. \nYou are equipped with an internet search tool and a special vectorstore of information about agents prompt engineering and adversarial attacks. \nIf the query covers the topics of agents, prompt engineering or adversarial attacks, use the vectorstore search.\n",
 'output': "I am an AI chatbot, so I don't have feelings as such, but I am functioning as intended. How can I help you today?"}

**Let's ask a question that requires multiple tools and self-reflection.**

In [12]:
agent_executor.invoke(
    {
        "input": "Tell me everything you know about undercover agents. If the info retrieved using some tools isn't satisfying, use more tools.",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...

I will search for 'undercover agents' in the vectorstore and write an answer based on the results.
{'tool_name': 'vectorstore_search', 'parameters': {'query': 'undercover agents'}}
inquired about current trends in anticancer drug discovery;
selected a target;
requested a scaffold targeting these compounds;
Once the compound was identified, the model attempted its synthesis.

They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test set containing a list of known chemical weapon agents and asked the agent to synthesize them. 4 out of 11 requests (36%) were accepted to obtain a synthesis solution and the agent attempted to consult documentation to execute the procedure. 7 out of 11 were rejected and among these 7 rejected cases, 5 happened after a Web search while 2 were rejected based on prompt only.
Generative Agents Simulation#
Generative Agents (Park, et al. 2023) is super fun experiment where 25 virtua

{'input': "Tell me everything you know about undercover agents. If the info retrieved using some tools isn't satisfying, use more tools.",
 'preamble': "\nYou are an expert who answers the user's question with the most relevant datasource. \nYou are equipped with an internet search tool and a special vectorstore of information about agents prompt engineering and adversarial attacks. \nIf the query covers the topics of agents, prompt engineering or adversarial attacks, use the vectorstore search.\n",
 'output': 'Undercover agents are professionals who use an alternate identity to provide detective work for a special assignment. They may engage in criminal activities as part of their investigation. \n\nUndercover operations present a range of challenges and risks, including the constant threat of exposure, balancing an undercover identity with a personal life, and the emotional and psychological toll on agents. \n\nPreparation is paramount for going undercover. The first rule is research